In [ ]:
import pandas as pd
import plotly
import plotly.express as px
import urllib.request
import json

In [ ]:
!pip install plotly==4.13.0

In [ ]:
print(plotly.__version__)

In [ ]:
df_mortes_ufocor = pd.read_csv('df_mortes_ufocor.csv')

df_mortes_munocor = pd.read_csv('df_mortes_munocor.csv')

In [ ]:
df_mortes_ufocor.head()

In [ ]:
df_mortes_ufocor_melted = df_mortes_ufocor.melt(id_vars=['Cód', 'Unidade da Federação', 'lat', 'lon'], var_name='Ano', value_name='Mortes')

df_mortes_ufocor_melted

In [ ]:
df_mortes_munocor_melted = df_mortes_munocor.melt(id_vars=['Cód', 'Município', 'lat', 'lon'], var_name='Ano', value_name='Mortes')

df_mortes_munocor_melted

In [ ]:
fig1 = px.scatter_mapbox(df_mortes_munocor_melted,
                        lat='lat',
                        lon='lon',
                        center={'lat': -15.79, 'lon': -47.88},
                        hover_name='Município',
                        size='Mortes',
                        size_max=40,
                        animation_frame='Ano',
                        color=[1 for i in range(61270)],
                        color_continuous_scale='Reds',
                        mapbox_style='open-street-map',
                        zoom=3,
                        height=700,
                        )

fig1.update_layout(coloraxis_showscale=False)

fig1.update_layout(title_text='Mortes no Brasil por município de ocorrência',
                  title_font_size=20,
                  title_x=0.5,
                  hoverlabel=dict(
                  bgcolor='black',    
                  font_size=16,        
                  font_family='Arial')
                  )

fig1.update_layout(margin=dict(l=150, r=150, t=75, b=0))

fig1['layout']['sliders'][0]['currentvalue']['prefix'] = 'Ano: '
fig1.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1000

fig1.show()

In [ ]:
df_causas = pd.read_csv('df_causas.csv')

df_causas.head()

In [ ]:
# pip install raceplotly

In [ ]:
from raceplotly.plots import barplot

my_raceplot = barplot(df_causas,
                      item_column='Descricao',
                      value_column='Quant',
                      time_column='Ano')

fig2 = my_raceplot.plot(title = 'Causas de mortes mais frequentes no Brasil entre 2008 e 2018<br>por categoria do CID-10',
                        item_label = 'Causas',
                        value_label = 'Número de mortes',
                        frame_duration = 1500)

fig2.update_layout(xaxis=dict(range=[0, 100000]),
                   height=700,
                   title_font_size=20,
                   title_x=0.5)

for num in range(11):
        fig2['frames'][num]['layout']['xaxis']['range'] = [0, 100000]

fig2['layout']['sliders'][0]['currentvalue']['prefix'] = 'Ano: '

fig2.update_layout(margin=dict(l=150, r=150, t=75, b=0))
        
fig2.show()

In [ ]:
# print(fig2)

In [ ]:
def read_geojson(url):
    with urllib.request.urlopen(url) as url:
        jdata = json.loads(url.read().decode())
    return jdata

estados = read_geojson('https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson')

df_suicidios = pd.read_csv('df_suicidios.csv')

df_suicidios.drop(columns=[str(x) for x in range(2008, 2019)], inplace=True)

df_suicidios.rename(columns={'tx2008':'2008', 'tx2009':'2009', 'tx2010':'2010', 'tx2011':'2011', 'tx2012':'2012', 'tx2013':'2013',
                             'tx2014':'2014', 'tx2015':'2015', 'tx2016':'2016', 'tx2017':'2017', 'tx2018':'2018'}, inplace=True)

df_suicidios.head()

In [ ]:
df_suicidios_melted = df_suicidios.melt(id_vars=['Cód', 'UF', 'lat', 'lon'], var_name='Ano', value_name='Taxa')

df_suicidios_melted.head()

In [ ]:
fig4 = px.choropleth_mapbox(df_suicidios_melted,
                            locations='UF',
                            geojson=estados,
                            center={'lat': -15.79, 'lon': -47.88},
                            featureidkey ='properties.name',
                            hover_name="UF",
                            color='Taxa',
                            color_continuous_scale='ylorbr',
                            animation_frame='Ano',
                            height=700,
                            mapbox_style='carto-darkmatter',
                            zoom=3,
                            range_color=[0, 13]
                            )

fig4.update_layout(title_text='Taxa de suicídios por 100 mil habitantes por estados',
                   title_font_size=20,
                   title_x=0.5,
                   )

fig4.update_layout(margin=dict(l=150, r=150, t=75, b=0))

fig4['layout']['sliders'][0]['currentvalue']['prefix'] = 'Ano: '
fig4.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1000

fig4.show()

In [ ]:
# pip install dash==1.17.0

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html

app = dash.Dash()

fig_names = ['Questão 1', 'Questão 2', 'Questão 3']
fig_dropdown = html.Div([
    dcc.Dropdown(
        id='fig_dropdown',
        options=[{'label': x, 'value': x} for x in fig_names],
        value=None
    )])
fig_plot = html.Div(id='fig_plot')
app.layout = html.Div([fig_dropdown, fig_plot])

@app.callback(
dash.dependencies.Output('fig_plot', 'children'),
[dash.dependencies.Input('fig_dropdown', 'value')])
def update_output(fig_name):
    return name_to_figure(fig_name)

def name_to_figure(fig_name):
    if fig_name == 'Questão 1':
        return dcc.Graph(figure=fig1)
    elif fig_name == 'Questão 2': 
        return dcc.Graph(figure=fig2)
    elif fig_name == 'Questão 3':
        return dcc.Graph(figure=fig4)

app.run_server(debug=True, use_reloader=False)